# Limpieza y Depuración de los Datos

En este notebook se realiza la lectura del dataset generado por scraping y se complementa la información del tipo de propiedad y municipios, así mismo se mejora la estructura del dataset (nombre de columnas y tipos de dato) para su posterior uso en el modelo de la iteación 1.

In [ ]:
!pip install pandas==1.2.4

In [ ]:
!pip install unidecode

In [ ]:
import time
import numpy as np
import pandas as pd

In [ ]:
# #Descargar datasets desde github
# !git clone https://github.com/andres-soto-h/monografia-udea-eacd.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/Seminario/DATASETS'
# !ls
df_propiedades=pd.read_csv('consolidado_propiedades_FR_12082021.csv', delimiter=';', encoding='utf-32') 
print(df_propiedades.shape)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/Seminario/DATASETS
(5311, 182)


In [ ]:
!ls

consolidado_propiedades_FR_12082021.csv    davipola.csv
consolidado_propiedades_FRv2_12082021.csv


In [ ]:
df_muni = pd.read_csv('davipola.csv', delimiter=',')
df_muni.shape

(1126, 5)

In [ ]:
df_propiedades.head(5)

,tipo,url,titulo,precio,area,habitaciones,banos,garajes,descripcion,antiguedad,Balcón,Baño Auxiliar,Comedor,Estudio,Servicios Públicos,Ventilación Natural,Zona de lavandería,En conjunto cerrado,Garaje / Parqueadero(s),Gimnasio,Piscina,Salón Comunal,Vivienda Multifamiliar,Zona Infantil,Zonas Verdes,Bombas de gasolina,Cerca centro comercial,Cómodas vias de acceso,Restaurantes,Seguridad,Supermercados / C.Comerciales,Trans. Público cercano,caractint,caractext,caractsec,otros_datos,latitud,longitud,Baño Independiente,Closet,...,Cocina de leña,Escalera de Emergencia,Gabinete de Incendios,Ascensor(es) inteligente(s),Industrial,Baños comunales,Baños Mixtos,Control de acceso digital,Servicios independientes,Vivienda Bifamiliar,Rociadores de agua,Finca ganadera,Corrales,Cuarto de conductores,Con casa club,Finca agrícola,Finca agroganadera,Cableado de Red,Puerta de seguridad,Puerta eléctrica,Mezzanine,Esquinero,Acceso para camiones,Sensor de movimiento,Auditorio,Patio Interno,Cuarto de servicio,Cuarto de Escoltas,Panorámica un lado,Cocineta,Finca avícola,Piso en Alfombra,Jaula de Golf,Con cerca eléctrica,Oficinas administrativas,Salón de videoconferencias,Pasaje Comercial,En club,Con casa prefabricada,Finca cafetera
0,Nueva,http://www.fincaraiz.com.co/vivari-de-san-nico...,VIVARI DE SAN NICOLAS San Nicolás - Rionegro,$ 512.262.940,"89,49 m2",3,3,1,"6568602 Vivari San Nicolás, es un conjunto res...",0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"['Balcón', 'Baño Auxiliar', 'Comedor', 'Estudi...","['En conjunto cerrado', 'Garaje / Parqueadero(...","['Bombas de gasolina', 'Cerca centro comercial...","{'AdvertId': '6568602', 'AdvertWASId': '656860...",6.147595,-75.379044,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Usada,https://www.fincaraiz.com.co/casa-en-venta/la-...,Casa en Venta La Ceja Jardines Del Camino Urba...,$ 534.000.000,"182,00 m²",4,4,2,Venta casa Urbanización Jardines del Camino ...,1 a 8 años,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,"['Balcón', 'Baño Auxiliar', 'Baño Independient...","['Acceso Pavimentado', 'Canchas Deportivas', '...","['Cerca de Zona Urbana', 'Parques cercanos', '...","{'AdvertId': '6386948', 'AdvertWASId': '638694...",6.022534,-75.421593,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Usada,https://www.fincaraiz.com.co/casa-en-venta/la-...,Casa en Venta La Ceja Jardines Del Camino Urba...,$ 534.000.000,"182,00 m²",4,4,2,Venta casa Urbanización Jardines del Camino ...,1 a 8 años,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,"['Balcón', 'Baño Auxiliar', 'Baño Independient...","['Acceso Pavimentado', 'Canchas Deportivas', '...","['Cerca de Zona Urbana', 'Parques cercanos', '...","{'AdvertId': '6386948', 'AdvertWASId': '638694...",6.022534,-75.421593,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Usada,https://www.fincaraiz.com.co/finca-en-venta/gu...,Finca en Venta Guarne guarne,$ 270.000.000,"4.070,00 m²",2,1,9,Casa campestre en jurisdicción del municipio d...,1 a 8 años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],['Terraza'],[],"{'AdvertId': '6575226', 'AdvertWASId': '0', 'A...",6.237039,-75.387001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Usada,https://www.fincaraiz.com.co/finca-en-venta/gu...,Finca en Venta Guarne guarne,$ 270.000.000,"4.070,00 m²",2,1,9,Casa campestre en jurisdicción del municipio d...,1 a 8 años,NaN,NaN,NaN

### Lectura de datos y cruce de información

In [ ]:
# df_propiedades=pd.read_csv('/content/monografia-udea-eacd/consolidado_propiedades_FR_12082021.csv', delimiter=';', encoding='latin1') 
# print(df_propiedades.shape)

# df_muni = pd.read_csv('/content/monografia-udea-eacd/davipola.csv', delimiter=',')
# df_muni.shape


In [ ]:
# Aplicar limpieza a los nombres de las columnas para quitar caracteres extraños y evitar errores
lista_columnas=list(df_propiedades.columns)
col_limpia=[]

import unidecode

for columna in lista_columnas:
  col=columna.lower().replace('/','').replace('.','').strip().replace('  ',' ').replace('  ',' ').replace(' ', '_')
  col=unidecode.unidecode(col)
  col_limpia.append(col)

columnas_nuevas=dict(zip(lista_columnas,col_limpia))

In [ ]:
df_propiedades.rename(columns=columnas_nuevas, inplace=True)
df_propiedades.rename(columns={'area':'area_m2'}, inplace=True)

In [ ]:
df_propiedades.head(1)

,tipo,url,titulo,precio,area_m2,habitaciones,banos,garajes,descripcion,antiguedad,balcon,bano_auxiliar,comedor,estudio,servicios_publicos,ventilacion_natural,zona_de_lavanderia,en_conjunto_cerrado,garaje_parqueadero(s),gimnasio,piscina,salon_comunal,vivienda_multifamiliar,zona_infantil,zonas_verdes,bombas_de_gasolina,cerca_centro_comercial,comodas_vias_de_acceso,restaurantes,seguridad,supermercados_ccomerciales,trans_publico_cercano,caractint,caractext,caractsec,otros_datos,latitud,longitud,bano_independiente,closet,...,cocina_de_lena,escalera_de_emergencia,gabinete_de_incendios,ascensor(es)_inteligente(s),industrial,banos_comunales,banos_mixtos,control_de_acceso_digital,servicios_independientes,vivienda_bifamiliar,rociadores_de_agua,finca_ganadera,corrales,cuarto_de_conductores,con_casa_club,finca_agricola,finca_agroganadera,cableado_de_red,puerta_de_seguridad,puerta_electrica,mezzanine,esquinero,acceso_para_camiones,sensor_de_movimiento,auditorio,patio_interno,cuarto_de_servicio,cuarto_de_escoltas,panoramica_un_lado,cocineta,finca_avicola,piso_en_alfombra,jaula_de_golf,con_cerca_electrica,oficinas_administrativas,salon_de_videoconferencias,pasaje_comercial,en_club,con_casa_prefabricada,finca_cafetera
0,Nueva,http://www.fincaraiz.com.co/vivari-de-san-nico...,VIVARI DE SAN NICOLAS San Nicolás - Rionegro,$ 512.262.940,"89,49 m2",3,3,1,"6568602 Vivari San Nicolás, es un conjunto res...",0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"['Balcón', 'Baño Auxiliar', 'Comedor', 'Estudi...","['En conjunto cerrado', 'Garaje / Parqueadero(...","['Bombas de gasolina', 'Cerca centro comercial...","{'AdvertId': '6568602', 'AdvertWASId': '656860...",6.147595,-75.379044,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_muni_antioquia=df_muni[df_muni["Nombre Departamento"]=='ANTIOQUIA']
df_muni_antioquia.head(1)

,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio,Tipo: Municipio / Isla / Área no municipalizada
0,05,5001.0,ANTIOQUIA,MEDELLÍN,Municipio


In [ ]:
#Aplicar transformación a los datos de municipios para usar el nombre como id de referencia en ambos datasets
df_muni_antioquia["Nombre Municipio"]=df_muni_antioquia["Nombre Municipio"].apply(lambda x: unidecode.unidecode(x.lower()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
list_muni=np.array(df_muni_antioquia["Nombre Municipio"])
list_muni=np.append(list_muni, np.array(['ceja','viboral','llano','llanogrande','pereira','union','unión','santuario']))

In [ ]:
#Funcion para extraer la ubicación de la propiedad a partir de los campos del dataset
def extract_ubicacion(lineas):
  texto=np.array(lineas)[0]
  palabras=texto.lower().replace(',','').replace(';','').split(' ')
  muni=''
  
  for palabra in palabras:

    if palabra in list_muni:
        muni = palabra
        break
    else:
        muni = ''

  if muni == '':
    texto=np.array(lineas)[1]
    palabras=texto.lower().replace(',','').replace(';','').split(' ')
  
    for palabra in palabras:

      if palabra in list_muni:
          muni = palabra
          break
      else:

          muni = ''


  return muni

In [ ]:
#Funcion para extraer el tipo de propiedad a partir de los campos del dataset
tipo_propiedades=['casa','finca','apartamento','apartaestudio','lote', 'apartamentos']

def extract_tipo(lineas):

  texto=np.array(lineas)[0]

  palabras=texto.lower().split(' ')
  tipo_prop=''
  
  for palabra in palabras:

    if palabra in tipo_propiedades:
        tipo_prop = palabra
        break
    else:
        tipo_prop = ''

  if tipo_prop == '':

    texto=np.array(lineas)[1]

    palabras=texto.lower().split(' ')
    tipo_prop=''
    
    for palabra in palabras:

      if palabra in tipo_propiedades:
          tipo_prop = palabra
          break
      else:
          #tipo de propiedad por defecto será apartamento
          tipo_prop = 'apartamento'

  return tipo_prop

In [ ]:
df_propiedades["municipio"]=df_propiedades[["titulo","descripcion"]].apply(extract_ubicacion,axis=1)

In [ ]:
df_propiedades["tipo_propiedad"]=df_propiedades[["titulo","descripcion"]].apply(extract_tipo, axis=1)

In [ ]:
df_propiedades.describe(include=['O'])

,tipo,url,titulo,precio,area_m2,habitaciones,banos,garajes,descripcion,antiguedad,caractint,caractext,caractsec,otros_datos,estrato,municipio,tipo_propiedad
count,5311,5311,5311,5311,5311,5311,5311,5311,5311,4167,5311,5311,5311,5311,5150,5311,5311
unique,2,2710,1140,443,667,16,14,12,2559,6,1747,1612,542,2619,7,11,5
top,Usada,https://www.fincaraiz.com.co/apartamento-en-ve...,Apartamento en Venta Rionegro RIONEGRO,$ 320.000.000,"60,00 m²",3,2,1,6519565 4 TORRES DE APTOS DE 15 PISOS - 6 APTO...,1 a 8 años,[],[],[],"{'AdvertId': '6519565', 'AdvertWASId': '651956...",4,rionegro,casa
freq,5202,2,162,120,110,2238,1994,2377,13,2198,561,747,759,13,1978,2652,2294


In [ ]:
df_propiedades.info(show_counts=True, max_cols=190)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5311 entries, 0 to 5310
Data columns (total 184 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    tipo                         5311 non-null   object 
 1    url                          5311 non-null   object 
 2    titulo                       5311 non-null   object 
 3    precio                       5311 non-null   object 
 4    area_m2                      5311 non-null   object 
 5    habitaciones                 5311 non-null   object 
 6    banos                        5311 non-null   object 
 7    garajes                      5311 non-null   object 
 8    descripcion                  5311 non-null   object 
 9    antiguedad                   4167 non-null   object 
 10   balcon                       2563 non-null   float64
 11   bano_auxiliar                2255 non-null   float64
 12   comedor                      675 non-null    float64
 13   e

### Realizar limpieza de datos y llenar nulos

In [ ]:
df_propiedades['municipio'].value_counts()

rionegro       2652
retiro          965
ceja            716
guarne          406
marinilla       220
viboral         136
llano            94
pereira          77
santuario        30
unión             8
llanogrande       7
Name: municipio, dtype: int64

In [ ]:
df_propiedades['tipo_propiedad'].value_counts()

casa             2294
apartamento      1734
finca            1154
apartaestudio      76
apartamentos       53
Name: tipo_propiedad, dtype: int64

In [ ]:
df_propiedades['tipo'].value_counts()

Usada    5202
Nueva     109
Name: tipo, dtype: int64

In [ ]:
list(df_propiedades[df_propiedades['tipo_propiedad']==""]["url"])

[]

In [ ]:
#unificar apartamentos
df_propiedades['tipo_propiedad']=df_propiedades['tipo_propiedad'].apply(lambda x: x.replace("apartamentos", "apartamento"))


In [ ]:
df_propiedades['tipo_propiedad'].value_counts()

casa             2294
apartamento      1787
finca            1154
apartaestudio      76
Name: tipo_propiedad, dtype: int64

In [ ]:
def unifica_muni(texto):
  muni=''
  
  if texto=='retiro':
    muni="el retiro"

  elif texto=='ceja':
    muni="la ceja"
 
  elif texto=='viboral':
    muni="el carmen de viboral"
  
  elif texto=='llano':
    muni="llanogrande"

  elif texto=='unión':
    muni="la unión"

  elif texto=='pereira':
    muni="san antonio de pereira"

  else:
    return texto

  return muni

In [ ]:
df_propiedades['municipio']=df_propiedades['municipio'].apply(unifica_muni)

In [ ]:
df_propiedades['municipio'].value_counts()

rionegro                  2652
el retiro                  965
la ceja                    716
guarne                     406
marinilla                  220
el carmen de viboral       136
llanogrande                101
san antonio de pereira      77
santuario                   30
la unión                     8
Name: municipio, dtype: int64

In [ ]:
#Llenar valores nulos con cero
df_propiedades=df_propiedades.fillna(0)

In [ ]:
df_propiedades = df_propiedades[df_propiedades['precio']!='A consultar']
df_propiedades.shape

(5307, 184)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [ ]:
#Transformar variable precio
df_propiedades['precio']=df_propiedades['precio'].apply(lambda x: x.replace('$','').replace('.','').replace(',','.').strip()).astype(float)

In [ ]:
#transformar variable área
df_propiedades['valida_ha']=df_propiedades['area_m2'].apply(lambda x: 1 if x.find("Ha")!=-1 else 0)

In [ ]:
#Limpiar texto y convertir a número
df_propiedades['area_m2'] = df_propiedades['area_m2'].apply(lambda x: x.replace('Ha','').replace('m2','').replace('m²','').replace('.','').replace(',','.').strip()).astype(float)

In [ ]:
#Corregir unidades hectareas
df_propiedades['area_m2'] = df_propiedades[['area_m2','valida_ha']].apply(lambda x: x.area_m2 if x.valida_ha == 0 else x.area_m2*10000,axis=1)

In [ ]:
df_propiedades.drop(['valida_ha'], axis=1, inplace=True)

In [ ]:
#Transformar variable habitaciones -- > El número por defecto será 1 sino especifica
df_propiedades['habitaciones']=df_propiedades['habitaciones'].apply(lambda x: x.replace('---','1').replace('Sin especificar','1').strip()).astype(float)

In [ ]:
#Transformar variable baños -- > El número por defecto será 1 sino especifica
df_propiedades['banos']=df_propiedades['banos'].apply(lambda x: x.replace('---','1').replace('Sin especificar','1').strip()).astype(float)

In [ ]:
#Transformar variable garajes -- > El número por defecto será 0 sino especifica
df_propiedades['garajes']=df_propiedades['garajes'].apply(lambda x: x.replace('---','0').replace('Sin especificar','0').replace('Más de 10','10').strip()).astype(float)

In [ ]:
df_propiedades['habitaciones'].value_counts()

3     2238
4     1254
2      924
5      454
1      239
6      136
7       34
8       10
13       4
10       4
15       2
9        2
21       2
12       2
14       2
Name: habitaciones, dtype: int64

In [ ]:
df_propiedades.info(show_counts=True, max_cols=190)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5307 entries, 0 to 5310
Data columns (total 184 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    tipo                         5307 non-null   object 
 1    url                          5307 non-null   object 
 2    titulo                       5307 non-null   object 
 3    precio                       5307 non-null   float64
 4    area_m2                      5307 non-null   float64
 5    habitaciones                 5307 non-null   float64
 6    banos                        5307 non-null   float64
 7    garajes                      5307 non-null   float64
 8    descripcion                  5307 non-null   object 
 9    antiguedad                   5307 non-null   object 
 10   balcon                       5307 non-null   float64
 11   bano_auxiliar                5307 non-null   float64
 12   comedor                      5307 non-null   float64
 13   e

In [ ]:
df_propiedades['precio'].describe()

count            5307
mean       2191793776
std       54800488515
min          95000000
25%         340000000
50%         550000000
75%        1200000000
max     2800000000000
Name: precio, dtype: float64

In [ ]:
# Eliminar del dataset propiedades con precios erroneos o posibles outliers
df_propiedades=df_propiedades[df_propiedades["precio"]<=2500000000]

In [ ]:
df_propiedades.shape

(4861, 184)

In [ ]:
df_propiedades = df_propiedades.drop_duplicates()
df_propiedades.shape

(2485, 184)

In [ ]:
df_propiedades.to_csv('df_prop_clean_12082021.csv', header=True, index=False, sep=";")